In [17]:
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and normalize the input data
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Convert labels to one-hot encoded format
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Create the model
model = Sequential()
model.add(Flatten(input_shape=(28, 28, 1)))
model.add(Dense(512, activation='relu', input_shape=(28, 28, 1), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(train_images)

# Train the model
model.fit(datagen.flow(train_images, train_labels, batch_size=32),
          steps_per_epoch=len(train_images) // 32, epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model.save('model/mnist_v3.h5')

Epoch 1/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.6385 - accuracy: 0.8771
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.3798 - accuracy: 0.9266
Epoch 3/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.3441 - accuracy: 0.9328
Epoch 4/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.3283 - accuracy: 0.9369
Epoch 5/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3189 - accuracy: 0.9372
Epoch 6/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.3097 - accuracy: 0.9404
Epoch 7/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.3029 - accuracy: 0.9400
Epoch 8/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2987 - accuracy: 0.9403
Epoch 9/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2934 - accuracy: 0.9413
Epoch 10/10
313/313 [==============================] - 1s 2ms/st

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import glob
from keras.models import load_model


model = load_model('model/mnist_v3.h5')

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(28, 28), color_mode="grayscale")
    img_array = np.array(img).reshape(-1, 28, 28, 1) 
    img_array = img_array.astype('float32') / 255.0
    return img_array


# Preprocess all the images
image_files = glob.glob('data/*.JPG')  
images = np.vstack([preprocess_image(img_file) for img_file in image_files])

predictions = model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

# Print the filename and corresponding predicted class for each image
for img_file, pred_class in zip(image_files, predicted_classes):
    print(f"File: {img_file}, Predicted Class: {pred_class}")

1/1 [==============================] - 0s 50ms/step
File: data/zero.JPG, Predicted Class: 0
File: data/two.JPG, Predicted Class: 8
File: data/one.JPG, Predicted Class: 8
File: data/nine.JPG, Predicted Class: 8
File: data/five.JPG, Predicted Class: 8
File: data/4.JPG, Predicted Class: 9
File: data/1.JPG, Predicted Class: 8
File: data/5.JPG, Predicted Class: 8
File: data/eight.JPG, Predicted Class: 8
File: data/six.JPG, Predicted Class: 0
File: data/three.JPG, Predicted Class: 3
File: data/8.JPG, Predicted Class: 8
File: data/2.JPG, Predicted Class: 2
File: data/four.JPG, Predicted Class: 0
File: data/7.JPG, Predicted Class: 2
File: data/6.JPG, Predicted Class: 0
File: data/3.JPG, Predicted Class: 3
File: data/9.JPG, Predicted Class: 8
File: data/0.JPG, Predicted Class: 0
File: data/seven.JPG, Predicted Class: 2
